In [2]:
import pathlib
import os
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(pathlib.Path(os.path.dirname(os.path.abspath(""))).resolve()))
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import DESCNUpliftModelFactory
from src.models.NNUpliftModeling.DESCNUpliftModel import DESCNUpliftModel

/home/ogrobertino/test_env/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
Failed to import duecredit due to No module named 'duecredit'


In [4]:
ds_name = 'lazada_v2'
features_percent = 100
train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)

In [5]:
import pandas as pd
ds = pd.read_csv(train_path, sep='\t')

In [6]:
input_size = len(ds.columns) - 2
input_size

83

In [47]:
params = {
    'input_dim': input_size,             # Должно быть задано в соответствии с данными
    'share_dim': [512], # Варианты размерности общих слоев
    'base_dim': [256],   # Варианты размерности базовых слоев
    'do_rate': [0.1],  # Варианты dropout
    'batch_norm': True, # Использование BatchNorm
    'normalization': 'divide', # Тип нормализации
    'prpsy_w': 0.5,   # Вес для потери пропенсити
    'escvr1_w': 0.5,  # Вес для потери ESTR
    'escvr0_w': 1.0,  # Вес для потери ESCR
    'mu1hat_w': 1.0,  # Вес для перекрестной потери TR
    'mu0hat_w': 0.5,  # Вес для перекрестной потери CR
    'batch_size': 128,
    'gradient_accumulation_steps' : 4,
    'optimizer': 
        {
            'name': 'Adam',
            'lr': 0.005,
            'weight_decay': 0.001
        },
    'epochs': 5
}

In [7]:
# params = {
#     'input_dim': input_size,             # Должно быть задано в соответствии с данными
#     'share_dim': [2], # Варианты размерности общих слоев
#     'base_dim': [2],   # Варианты размерности базовых слоев
#     'do_rate': [0.1, 0.2, 0.3],  # Варианты dropout
#     'batch_norm': [True, False], # Использование BatchNorm
#     'normalization': ['none', 'divide'], # Тип нормализации
#     'factual_loss_weight': [0.8, 1.0, 1.2], # Вес фактической потери
#     'propensity_loss_weight': [0.05, 0.1, 0.2], # Вес потери пропенсити
#     'tau_loss_weight': [0.05, 0.1, 0.2],    # Вес потери tau (если применимо)
#     'batch_size': 32,
#     'gradient_accumulation_steps' : 2,
#     'epochs': 1
# }

In [48]:
configs = DESCNUpliftModel.generate_config(count=4, params = params)

In [49]:
factory = DESCNUpliftModelFactory
config = configs[0]
batch_size=32
max_size=100000

In [50]:
model, train, test = factory.create(config, train_path, test_path)

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [51]:
model.num_params()

1227690

In [11]:
# from torch.utils.data import DataLoader
# dl = DataLoader(
#     train, 
#     batch_size=32, 
#     shuffle=True,

#     num_workers=0
# )

In [12]:
# kek = model.predict(train)

In [13]:
# pd.DataFrame(kek['uplift'])

In [14]:
# q = model.predict(train)

In [15]:
# q['df'][q['df']['treatment'] == 1]

In [43]:
train_test_model(ds_name='lazada_v2', features_percent=100, factory=DESCNUpliftModelFactory, config=configs[0], NN=True)   

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5 - 20.0% - Loss: 0.4884
Epoch 1/5 - 40.0% - Loss: 0.4690
Epoch 1/5 - 60.0% - Loss: 0.4638
Epoch 1/5 - 79.9% - Loss: 0.4574
Epoch 1/5 - 99.9% - Loss: 0.4541
Validation after epoch
Epoch 1/5, Train Loss: 0.4540, Val Loss: 0.4371, Val AUUC: -0.0032, LR: 0.005000
Epoch 2/5 - 20.0% - Loss: 0.4411
Epoch 2/5 - 40.0% - Loss: 0.4408
Epoch 2/5 - 60.0% - Loss: 0.4410
Epoch 2/5 - 79.9% - Loss: 0.4413
Epoch 2/5 - 99.9% - Loss: 0.4413
Validation after epoch
Epoch 2/5, Train Loss: 0.4412, Val Loss: 0.4385, Val AUUC: -0.0016, LR: 0.005000
Epoch 3/5 - 20.0% - Loss: 0.4478
Epoch 3/5 - 40.0% - Loss: 0.4452
Epoch 3/5 - 60.0% - Loss: 0.4435
Epoch 3/5 - 79.9% - Loss: 0.4422
Epoch 3/5 - 99.9% - Loss: 0.4412
Validation after epoch
Epoch 3/5, Train Loss: 0.4411, Val Loss: 0.4356, Val AUUC: -0.0017, LR: 0.005000
Epoch 4/5 - 20.0% - Loss: 0.4442
Epoch 4/5 - 40.0% - Loss: 0.4389
Epoch 4/5 - 60.0% - Loss: 0.4388
Epoch 4/5 - 79.9% - Loss: 0.4405
Epoch 4/5 - 99.9% - Loss: 0.4404
Validation after epoch
Epoch

cfg:{'lr': 0.001, 'decay_rate': 0.95, 'decay_step_size': 1, 'l2': 0.001, 'model_name': 'DESCN_128_64_20220215_175927', 'n_experiments': 5, 'batch_size': 5000, 'share_dim': 128, 'base_dim': 64, 'reweight_sample': 1, 'val_rate': 0.2, 'do_rate': 0.1, 'normalization': 'divide', 'epochs': 5, 'log_step': 50, 'pred_step': 1, 'optim': 'Adam', 'BatchNorm1d': 'true', 'prpsy_w': 0.5, 'escvr1_w': 0.5, 'escvr0_w': 1, 'h1_w': 0, 'h0_w': 0, 'mu0hat_w': 0.5, 'mu1hat_w': 1, 'imb_dist': 'wass', 'imb_dist_w': 0,

In [ ]:
# for i in tqdm(range(len(configs)), desc="Training models"):
#     train_test_model(ds_name='lazada', features_percent=100, factory=DRModelFactory, config=configs[i])   

In [44]:
from src.global_params import BASE_PATH
path = pd.read_csv(BASE_PATH + "/exps2/stats.tsv", sep='\t').Path.iloc[-1]
model = DESCNUpliftModel(from_load=True, path = str(BASE_PATH) + "/" + path)

/home/ogrobertino/UpliftModelingResearch/src/models/NNUpliftModeling/INNUpliftModeling.py:352: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, ma

In [46]:
model.num_params()

85866

In [18]:
path

'exps2/lazada_v2/100/9'

In [19]:
str(BASE_PATH) + "/" + path

'/home/ogrobertino/UpliftModelingResearch/exps2/lazada_v2/100/9'

In [12]:
res = pd.read_csv(str(BASE_PATH) + "/" + path + "/predictions.tsv", sep='\t')

In [21]:
model.config

{'batch_size': 64,
 'epochs': 5,
 'early_stopping_patience': 2,
 'optimizer': {'name': 'Adam', 'lr': 0.005, 'weight_decay': 0.001},
 'scheduler': {'name': 'ReduceLROnPlateau', 'patience': 1, 'factor': 0.5},
 'use_gpu': True,
 'num_workers': 0,
 'inference_batch_size': 32,
 'input_dim': 83,
 'share_dim': 32,
 'base_dim': 32,
 'do_rate': 0.1,
 'batch_norm': True,
 'normalization': 'divide',
 'prpsy_w': 0.5,
 'escvr1_w': 0.5,
 'escvr0_w': 1.0,
 'mu1hat_w': 1.0,
 'mu0hat_w': 0.5,
 'gradient_accumulation_steps': 2}

In [15]:
from src.metric import get_auuc

In [23]:
get_auuc(res)

-0.0035888787886069313

In [24]:
res2 = res.copy(deep=True)
res2['score'] = -1 * res2['score']

In [25]:
get_auuc(res2)

-0.04996016733696808

In [26]:
# from torch.utils.data import DataLoader
# dl = DataLoader(
#     test, 
#     batch_size=32, 
#     shuffle=True,

#     num_workers=0
# )

In [13]:
preds = model.predict(test)

In [14]:
preds.keys()

dict_keys(['uplift', 'df', 'p_estr', 'p_escr', 'p_prpsy'])

In [17]:
get_auuc(preds["df"])

-0.0035888787886069313

In [21]:
df = preds["df"]

In [25]:
new1 = preds["p_estr"] - preds["p_escr"]
new1 = new1[:, 0]

In [26]:
new1_df = df.copy(deep=True)
new1_df["score"] = new1

In [27]:
get_auuc(new1_df)

0.010982155682655821

In [29]:
new2 = preds["p_estr"] * preds["p_prpsy"] - preds["p_escr"] * (1 - preds["p_prpsy"])
new2 = new2[:, 0]

new2_df = df.copy(deep=True)
new2_df["score"] = new2

In [30]:
get_auuc(new2_df)

0.012947675721376155

In [31]:
from src.metric import get_auuc, get_qini, uplift_by_percentile_CUM

In [34]:
get_auuc(new2_df), get_auuc(df)

(0.012947675721376155, -0.0035888787886069313)

In [35]:
get_qini(new2_df), get_qini(df)

(0.46074126068345955, 0.3758347256505864)

In [36]:
from src.global_params import COL_TARGET, COL_TREATMENT

In [37]:
uplift_by_percentile_CUM(df[COL_TARGET], df["score"],
                                                df[COL_TREATMENT], strategy='overall', bins=20)

,n_treatment,n_control,response_rate_treatment,response_rate_control,uplift
percentile,,,,,
5,11409,174,0.134718,0.137931,-0.003213
10,21675,1491,0.098547,0.061033,0.037514
15,29645,5104,0.083792,0.043299,0.040492
20,35283,11049,0.075050,0.032401,0.042649
25,39262,18653,0.069762,0.025948,0.043815
30,42232,27266,0.066040,0.022996,0.043044
35,44361,36720,0.063592,0.020125,0.043467
40,46074,46590,0.061814,0.018738,0.043076
45,47362,56884,0.060428,0.017228,0.043200


In [38]:
uplift_by_percentile_CUM(new2_df[COL_TARGET], new2_df["score"],
                                                new2_df[COL_TREATMENT], strategy='overall', bins=20)

,n_treatment,n_control,response_rate_treatment,response_rate_control,uplift
percentile,,,,,
5,11403,180,0.134877,0.138889,-0.004012
10,21648,1518,0.097977,0.057971,0.040006
15,29580,5169,0.083469,0.041788,0.041681
20,35021,11311,0.075298,0.031739,0.043559
25,35125,22790,0.075103,0.017376,0.057727
30,35259,34239,0.074846,0.013289,0.061557
35,35444,45637,0.074512,0.010868,0.063644
40,35695,56969,0.074100,0.009654,0.064446
45,36046,68200,0.073545,0.009208,0.064337


In [ ]:
    auqc_model = get_qini(predicted)
    compressions = compressions or {}
    uplift_precision = uplift_by_percentile_CUM(predicted[COL_TARGET], predicted["score"],
                                                predicted[COL_TREATMENT], strategy='overall', bins=20)

In [28]:
preds['y0']

array([[-4.9499893],
       [-4.6410084],
       [-4.771675 ],
       ...,
       [-4.5801687],
       [-4.603956 ],
       [-4.2976193]], dtype=float32)

In [29]:
preds['y1']

array([[-4.5946   ],
       [-4.2405596],
       [-4.392601 ],
       ...,
       [-4.211261 ],
       [-4.228192 ],
       [-3.8930411]], dtype=float32)

In [35]:
preds['uplift']

array([[-0.8277783 ],
       [-0.32938668],
       [-0.37059146],
       ...,
       [-0.577477  ],
       [-0.82533956],
       [-0.6777634 ]], dtype=float32)